# RNN Sequence Prediction
Try to guess the key

In [5]:
import pickle
import os
import random
import tensorflow as tf
from tensorflow.contrib import rnn
import matplotlib.pyplot as plt
import numpy as np
import time

%matplotlib inline

# TF settings
logs_path = os.getcwd()
writer = tf.summary.FileWriter(logs_path)

In [6]:
# Load data
with open("training_data.pkl", 'rb') as f:
    data = pickle.load(f)

print(data)    

{'sbox': {0: 0, 1: 1, 2: 2, 3: 3}, 'x_train': array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [0, 1],
       [1, 0],
       [2, 3],
       [3, 2],
       [0, 2],
       [1, 3],
       [2, 0],
       [3, 1],
       [0, 3],
       [1, 2],
       [2, 1],
       [3, 0],
       [0, 1],
       [1, 0],
       [2, 3],
       [3, 2],
       [0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [0, 3],
       [1, 2],
       [2, 1],
       [3, 0],
       [0, 2],
       [1, 3],
       [2, 0],
       [3, 1],
       [0, 2],
       [1, 3],
       [2, 0],
       [3, 1],
       [0, 3],
       [1, 2],
       [2, 1],
       [3, 0],
       [0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [0, 1],
       [1, 0],
       [2, 3],
       [3, 2],
       [0, 3],
       [1, 2],
       [2, 1],
       [3, 0],
       [0, 2],
       [1, 3],
       [2, 0],
       [3, 1],
       [0, 1],
       [1, 0],
       [2, 3],
       [3, 2],
       [0, 0],
       [1, 1],
       [2, 2],
       [3

In [40]:
# Clear graph
tf.reset_default_graph()

In [41]:
# Param
# Training
learning_rate = 0.001
training_steps = 10000
batch_size = 20
display_step = 200

# Network
n_input = 2
n_hidden = 512
n_output = 2
n_sequence = 1

# num classes this is proportional to 2**bit_size
n_classes = 2**2

# tf graph input
x = tf.placeholder("float", [None, n_input, n_sequence])
y = tf.placeholder("float", [None, n_output])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}

biases = {
    'out': tf.Variable(tf.random_normal([n_output], mean=1.0))
}


In [42]:
# RNN Model
def RNN(x, weights, biases):
    # Reshape input to [1, n_input]
    x = tf.reshape(x, [-1, n_input * n_sequence])

    # generate a n_input-element sequence of inputs
    x = tf.split(x, n_input, 1)

    # 1-layer LSTM with n_hidden units
    rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # 2-layer LSTM cell
    # rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden), rnn.BasicLSTMCell(n_hidden)])

    # Get lstm output
    _outputs, _states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # Linear Activation, using rnn inner loop last output
    # Return last two outputs
    pred = tf.matmul(_outputs[-1], weights['out']) + biases['out']
    
    return pred

pred = RNN(x, weights, biases)

In [43]:
# Loss and optimizer

# Cross entropy loss
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# Simple square loss
cost = tf.reduce_mean(tf.square(pred - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Model eval
#print(tf.shape(pred))
#print(tf.shape(y))
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize variables
init = tf.global_variables_initializer()

In [48]:
# Launch the graph
start = time.time()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    
    writer.add_graph(session.graph)
    
    while step < training_steps:
       # generate a minibatch. Add some randomness on selection
        
        symbols_in_keys = np.reshape(data['x_train'], [-1, n_input, n_sequence])
        
        symbols_out_onehot = np.reshape(np.zeros([n_output], dtype=float), [1, -1])
        
        _, acc, loss, onehot_pred = session.run(
            [optimizer, accuracy, cost, pred], 
            feed_dict={x: symbols_in_keys, y: symbols_out_onehot}) 
        
        loss_total += loss
        acc_total += acc
        
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                 "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                 "{:.2f}%".format(100*acc_total/display_step))
            
            acc_total = 0
            loss_total = 0
            data_in = [data['x_train'][i] for i in range(len(data['x_train']))]
            data_out = [data['y_train'][i] for i in range(len(data['y_train']))]
            data_out_pred = onehot_pred
            #symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("{} - [{}] vs [{}]".format(data_in, data_out, data_out_pred))
            
        step += 1
        
#     print("Optimization Finished")
#     print("Elapsed time: ", time.time() - start_time)
#     print("Run on command line.")
#     print("\ttensorboard --logdir=%s" & (logs_path))
#     print("Point your web browser to: http://localhost:6006/")
    
#     while True:
#         prompt = "%s input: " % n_input
#         sentence = input(prompt)
#         sentence = sentence.strip()
#         words = sentence.split(' ')
        
#         if len(n_classes) != n_input:
#             continue
        
#         try:
#             symbols_in_keys = [i for i in range(n_classes)]
            
#             for i in range(32):
#                 keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
#                 onehot_pred = session.run(pred, feed_dict={x: keys})
#                 onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
#                 sentence = "%s %s" % (setence, onehot_pred_index)
#                 symbols_in_keys = symbols_in_keys[1:]
#                 symbols_in_keys.append(onthot_pred_index)
#             print(sentence)
            
#         except:
#             print("word not in dict")

Iter= 200, Average Loss= 0.028104, Average Accuracy= 55.56%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2,

Iter= 600, Average Loss= 0.000000, Average Accuracy= 50.00%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2,

Iter= 1000, Average Loss= 0.000126, Average Accuracy= 51.81%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2

Iter= 1400, Average Loss= 0.000230, Average Accuracy= 50.06%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2

Iter= 1800, Average Loss= 0.000000, Average Accuracy= 50.00%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2

...

Iter= 8000, Average Loss= 0.000000, Average Accuracy= 50.00%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2

Iter= 8400, Average Loss= 0.000122, Average Accuracy= 49.06%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2

Iter= 8800, Average Loss= 0.000115, Average Accuracy= 49.97%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2

Iter= 9200, Average Loss= 0.000066, Average Accuracy= 48.72%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2

Iter= 9800, Average Loss= 0.000074, Average Accuracy= 49.50%
[array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 0]), array([1, 1]), array([2, 2]), array([3, 3]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 3]), array([1, 2]), array([2, 1]), array([3, 0]), array([0, 2]), array([1, 3]), array([2, 0]), array([3, 1]), array([0, 1]), array([1, 0]), array([2, 3]), array([3, 2]), array([0, 0]), array([1, 1]), array([2